# Neural network basics
----

### Package Version
- tensorflow==2.3.0
- scikit-learn==0.22.2.post1
- pandas==1.1.4

## Given a Bank customer, can we build a classifier which can determine whether they will leave or not using Neural networks?
The case study is from an open source dataset from Kaggle.

Link to the Kaggle project site:
https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

### Read the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

ds = pd.read_csv("/content/drive/MyDrive/Churn_Modelling.csv")

In [ ]:
ds.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


### Drop the columns which are unique for all users like IDs

In [ ]:
ds['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

### RowNumber, CustomerId, and Surname are unique hence we are dropping it

In [ ]:
ds = ds.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


### Distinguish the feature and target set

In [ ]:
X = ds.iloc[:,0:10].values # Credit Score through Estimated Salary
y = ds.iloc[:,10].values # Exited

### Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany. This will convert those strings into scalar values for analysis.

In [ ]:
from sklearn.preprocessing import LabelEncoder

print(X[:8,1], '... will now become: ')

label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1])
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


### We will do the same thing for gender. this will be binary in this dataset

In [ ]:
print(X[:6,2], '... will now become: ')

label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


### The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). Therefore, one way to get rid of that problem is to split the countries into respective dimensions. Gender does not need this as it is binary

### Converting the string features into their own dimensions. Gender doesn't matter here because its binary

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

transform = ColumnTransformer([("countries", OneHotEncoder(), [1])], remainder="passthrough") # 1 is the country column
X = transform.fit_transform(X)

In [ ]:
X.shape

(10000, 12)

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
# A 0 on two countries means that the country has to be the one variable which wasn't included 
# This will save us from the problem of using too many dimensions
X = X[:,1:] # Got rid of Spain as a dimension.

### Divide the data set into Train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into the Training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Normalize the train and test data

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Initialize & build the model

In [ ]:
from tensorflow.keras.models import Sequential

# Initializing the ANN
classifier = Sequential()

In [ ]:

from tensorflow.keras.layers import Dense

# The amount of nodes (dimensions) in hidden layer should be the average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [ ]:
#Add 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [ ]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [ ]:
# Create optimizer with default learning rate
# sgd_optimizer = tf.keras.optimizers.SGD()
# Compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2066 - accuracy: 0.7960 - val_loss: 0.1819 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.1741 - accuracy: 0.7960 - val_loss: 0.1681 - val_accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 979us/step - loss: 0.1664 - accuracy: 0.7960 - val_loss: 0.1640 - val_accuracy: 0.7975
Epoch 4/100
250/250 [==============================] - 0s 983us/step - loss: 0.1640 - accuracy: 0.7960 - val_loss: 0.1626 - val_accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 0s 995us/step - loss: 0.1631 - accuracy: 0.7960 - val_loss: 0.1620 - val_accuracy: 0.7975
Epoch 6/100
250/250 [==============================] - 0s 989us/step - loss: 0.1627 - accuracy: 0.7960 - val_loss: 0.1618 - val_accuracy: 0.7975
Epoch 7/100
250/250 [==============================] - 0s 981us/step - loss: 0.1625 - accuracy: 0.7960 - val_loss: 0.1616 - val_accura

### Predict the results using 0.5 as a threshold

In [ ]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.20408505]
 [0.20437309]
 [0.20437112]
 ...
 [0.2038745 ]
 [0.20379704]
 [0.20420441]]


In [ ]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


### Print the Accuracy score and confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test, y_pred)
print(cm1)

[[1595    0]
 [ 405    0]]


In [ ]:
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print (accuracy_model1, '% of testing data was classified correctly')

79.75 % of testing data was classified correctly


### Optimize the model

Some important parameters to look out for while optimizing neural networks are:

-Type of architecture

-Number of Layers

-Number of Neurons in a layer

-Regularization parameters

-Learning Rate

-Type of optimization / backpropagation technique to use

-Dropout rate

-Weight sharing

#### Number of Layers:
We will keep it similar to the above model so that we can compare the accuracy.
1 hidden layer.

#### Activation:
input layer: relu becasue we are in an input layer. uses the ReLu activation function for  ϕ
output layer: sigmoid becasue we are in an output layer. uses the Sigmoid activation function for  ϕ . This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.

#### Type of optimization / backpropagation technique to use: 
We will use Adam. Adam is a very efficeint variation of Stochastic Gradient Descent. For Adam and its variant, learning rate or the decay rate does not really matter too much.

#### Learning Rate:
default learning rate 0.001.

#### Number of Neurons in a layer:
We will keep it 6 as per our initial calculation above.

#### Weight sharing / kernel_initializer: 
uniform the distribution with which we randomly initialize weights for the nodes in this layer.

#### Loss:
loss: binary_crossentropy This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is Binary, it is binary_crossentropy. If Categorical, then it is called categorical_crossentropy

### Rebuilding the model using these optimised parameters

In [ ]:
# Initializing the ANN
classifier = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [ ]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

In [ ]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [ ]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6201 - accuracy: 0.7945 - val_loss: 0.5050 - val_accuracy: 0.7975
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4520 - accuracy: 0.7959 - val_loss: 0.4267 - val_accuracy: 0.7985
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.8124 - val_loss: 0.4117 - val_accuracy: 0.8325
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4090 - accuracy: 0.8231 - val_loss: 0.4027 - val_accuracy: 0.8285
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4012 - accuracy: 0.8270 - val_loss: 0.3954 - val_accuracy: 0.8335
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8279 - val_loss: 0.3895 - val_accuracy: 0.8345
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3905 - accuracy: 0.8310 - val_loss: 0.3849 - val_accuracy: 0.8335

### Predict the results using 0.5 as a threshold

In [ ]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.25750184]
 [0.36785206]
 [0.22174105]
 ...
 [0.22687352]
 [0.1534543 ]
 [0.28107512]]


In [ ]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


### Print the Accuracy score and confusion matrix

In [ ]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[1516   79]
 [ 199  206]]


In [ ]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

86.1 % of testing data was classified correctly


<img src="http://drive.google.com/uc?export=view&id=1tpOCamr9aWz817atPnyXus8w5gJ3mIts" width=500px>

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.